In [4]:
import csv
import pandas as pd
from glob import glob
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Prepare data

In [5]:
task_files = glob("task*.csv")

In [6]:
def format_data(data):
    result = []
    for i in data:
        r = [k.strip(",") for k in i.strip("\n").strip(" ").strip(",").split(":")]
        r[1] = float(r[1])
        result.append(r)
    return result

In [7]:
def prepare_data(task_files):
    tasks = {t: open(t) for t in task_files}
    raw_data = {t: v.readlines() for t, v in tasks.items()}
    data_dict = [{
        "file_name": n,
        "test_name": i[1].strip("\n").strip(".").strip("Test Name:,"),
        "data": format_data(i[2:])
    } for n, i in raw_data.items()]
    data_dict = sorted(data_dict, key=lambda x: x["file_name"])
    print("Test types:\n", set([i['test_name'] for i in data_dict]))
    print("")
    print("File Names:\n", list([i['file_name'] for i in data_dict]))
    return data_dict

In [8]:
data_dict = prepare_data(task_files)

Test types:
 {'4.12.0-standard', '4.12.0-config', '4.12.0-patch', '4.15.0-101-generic'}

File Names:
 ['task1-1.csv', 'task1-2.csv', 'task1-3.csv', 'task1-4.csv', 'task2-1.csv', 'task2-2.csv', 'task2-3.csv', 'task2-4.csv', 'task3-1.csv', 'task3-2.csv', 'task3-3.csv', 'task3-4.csv', 'task4-1.csv', 'task4-2.csv', 'task4-3.csv', 'task4-4.csv']


In [9]:
cols = [i[0] for i in data_dict[0]["data"]]

In [10]:
heat_map_data = [[j[1] for j in i["data"]] for i in data_dict]

In [11]:
relative_boost = [[v / heat_map_data[1][j] for j, v in enumerate(u)] for i, u in enumerate(heat_map_data)]

In [12]:
test_names = [i["test_name"] for i in data_dict]

In [13]:
file_name = [i["file_name"] for i in data_dict]

In [14]:
df = pd.DataFrame(heat_map_data, columns=cols)
df["test_names"] = test_names
df["file_name"] = file_name
df
absolute_df = df

In [15]:
df = pd.DataFrame(relative_boost, columns=cols)
df["test_names"] = test_names
df["file_name"] = file_name
df
relative_df = df

In [16]:
absolute_df.to_csv("result-absolute.csv")

In [17]:
relative_df.to_csv("result-relative.csv")

In [18]:
intask_df = [absolute_df[absolute_df["file_name"].str.contains(f"task{i}-")] for i in range(1,5)]

In [19]:
crosstask_df = [absolute_df[absolute_df["file_name"].str.contains(f"-{i}.csv")] for i in range(1,5)]

In [25]:
def get_numeric_cols(a):
    return a[[i for i in list(a.columns) if i not in ["test_names", "file_name"]]]

In [77]:
def get_normalized(a):
    b = get_numeric_cols(a)
    p = b[:1].sum()
    c = b.subtract(p).div(p).mul(100).round()
    return c

In [78]:
a = crosstask_df[0]

In [79]:
b = crosstask_df[1:]

In [80]:
a.T.head()

,0,4,8,12
ref kbest,2.8e-08,2.8e-08,2.8e-08,2.8e-08
ref average,4.4e-08,3.3e-08,3.2e-08,3.2e-08
cpu kbest,0.00308454,0.00308459,0.00308459,0.00308458
cpu average,0.00365277,0.00311658,0.00310848,0.00312007
getpid kbest,5.3e-07,7.7e-08,7.8e-08,8e-08


In [81]:
normalized = get_normalized(crosstask_df[0])

In [82]:
normalized.T.head()

,0,4,8,12
ref kbest,0.0,0.0,0.0,0.0
ref average,0.0,-25.0,-27.0,-27.0
cpu kbest,0.0,0.0,0.0,0.0
cpu average,0.0,-15.0,-15.0,-15.0
getpid kbest,0.0,-85.0,-85.0,-85.0


In [83]:
normalized_avg = normalized[
    [i for i in normalized.columns if i.find("average") >= 0]
]

In [135]:
? plt.colorbar

In [159]:
dpi = 1000
import matplotlib as mpl
mpl.rcParams['figure.dpi']= dpi

import matplotlib as mpl
mpl.rc("savefig", dpi=dpi)

In [160]:
plt.matshow(
    normalized_avg.T, 
#     cmap='hsv', 
    interpolation='nearest'
)
xlabels = [str(i) for i in [
    "task1-4.15.0",
    "task2-4.12.0",
    "task3-4.12.0-patch",
    "task4-4.12.0-config"
]]
plt.xticks(np.arange(len(xlabels)), xlabels, rotation='vertical')
ylabels = [i.split("average")[0].strip(" ") for i in normalized_avg.columns]
plt.yticks(np.arange(len(ylabels)), ylabels)
plt.colorbar()

for (i, j), z in np.ndenumerate(normalized_avg.T):
    plt.text(j, i, '{:0.0f}'.format(z), ha='center', va='center')

